In [2]:
import mysql.connector as connector
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

In [37]:
try:
    pool_b=MySQLConnectionPool(pool_name="little_lemon_pool",
                           pool_size=2,
                           host="localhost",
                           database="little_lemon_db",
                           user="quepatos",
                           password="10560714")
    print("The connection pool is created with a name: little_lemon_pool\nThe pool size is: 2")
except: 
    print("Wrong password or username, try again")

The connection pool is created with a name: little_lemon_pool
The pool size is: 2


In [38]:
guest1=pool_b.get_connection()
cursor1=guest1.cursor()
cursor1.execute('USE little_lemon_db')

In [39]:
insert_stm_1="""
INSERT INTO Bookings(BookingID,TableNo,GuestFirstName,GuestLastName,BookingSlot,EmployeeID)
VALUES
(7, 8,'Anees','Java','18:00',6)"""

In [26]:
cursor1.execute(insert_stm_1)

IntegrityError: 1062 (23000): Duplicate entry '7' for key 'bookings.PRIMARY'

In [20]:
guest1.commit()

In [40]:
guest2=pool_b.get_connection()
cursor2=guest2.cursor()
cursor2.execute('USE little_lemon_db')

In [41]:
insert_stm_2="""
INSERT INTO Bookings(BookingID, TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(8,5,'Bald','Vin','19:00',6)"""

In [29]:
cursor2.execute(insert_stm_2)

In [30]:
guest2.commit()

In [43]:
try: 
    guest3=pool_b.get_connection()
except:
    print("Not enough connections, adding one...")
    pool_b.add_connection()
    guest3=pool_b.get_connection()

In [44]:
cursor3=guest3.cursor()
cursor3.execute('USE little_lemon_db')

In [45]:
insert_stm_3="""
INSERT INTO Bookings(BookingID, TableNo, GuestFirstName,GuestLastName, BookingSlot, EmployeeID)
VALUES
(9,12,'Jay','Kon','19:30',6)"""

In [46]:
cursor3.execute(insert_stm_3)

In [47]:
guest3.commit()

In [48]:
guest1.close()

In [49]:
guest2.close()

In [52]:
try:
    guest3.close()
except:
    print("The 2 connections are available and another one cannot be added.")

The 2 connections are available and another one cannot be added.


In [60]:
create_report_1="""
SELECT CONCAT(EmployeeID, ' ', Name) AS 'Manager ID and name' FROM Employees WHERE EmployeeID=01;
"""
create_report_2="""
SELECT Role, Annual_Salary AS 'Highest salary' FROM Employees 
WHERE Annual_Salary=(SELECT MAX(Annual_Salary) from Employees);
"""
create_report_3="""
SELECT COUNT(BookingSlot) AS 'Number of bookings between 18:00 and 20:00' FROM Bookings WHERE BookingSlot>='18:00' and BookingSlot<='20:00';
"""
create_report_4="""
SELECT BookingID, CONCAT(GuestFirstName, ' ',GuestLastName) as 'Guests waiting for table' FROM Bookings WHERE EmployeeID=06 ORDER BY BookingSlot;
"""




In [54]:
connection=pool_b.get_connection()

In [55]:
cursor=connection.cursor()

In [56]:
cursor.execute(create_report_1)
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
print(results)

('Manager ID and name',)
[('1 Mario Gollini',)]


In [57]:
cursor.execute(create_report_2)
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
print(results)

('Role', 'Highest salary')
[('Manager', '$70,000')]


In [58]:
cursor.execute(create_report_3)
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
print(results)

('Number of bookings between 18:00 and 20:00',)
[(7,)]


In [61]:
cursor.execute(create_report_4)
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
print(results)

('BookingID', 'Guests waiting for table')
[(7, 'Anees Java'), (8, 'Bald Vin'), (9, 'Jay Kon')]


In [92]:
basic_sales_report_proc="""
CREATE PROCEDURE BasicSalesReport(OUT p1 INT, OUT p2 INT, OUT p3 INT, OUT p4 INT)
BEGIN
SELECT SUM(BillAmount) INTO p1 FROM Orders;
SELECT AVG(BillAmount) INTO p2 FROM Orders;
SELECT MIN(BillAmount) INTO p3 FROM Orders;
SELECT MAX(BillAmount) INTO p4 FROM Orders;
END"""

p1='SUM'
p2='AVG'
p3='MIN'
p4='MAX'

In [85]:
cursor.execute(basic_sales_report_proc)

ProgrammingError: 1304 (42000): PROCEDURE BasicSalesReport already exists

In [94]:
print(('SUM','AVG','MIN','MAX'))
cursor.callproc('BasicSalesReport',(p1,p2,p3,p4))

('SUM', 'AVG', 'MIN', 'MAX')


(243, 49, 37, 86)

In [95]:
connection.close()

In [97]:
guest1=pool_b.get_connection()
cursor1=guest1.cursor(buffered=True)

cursor1.execute("""
SELECT B.BookingID as 'bID', B.TableNo as 'tNo', CONCAT(GuestFirstName, ' ', GuestLastName) as 'GuestName',
B.BookingSlot as 'BS', B.EmployeeID as 'eID', E.Name as 'eName', E.Role as 'eRole', E.Adress as 'eAddres',
E.Contact_Number as 'eCN',  E.Email as 'eEmail', E.Annual_Salary as 'eAS'
FROM Bookings as B
LEFT JOIN Employees as E ON B.EmployeeID=E.EmployeeID;""")
                

In [99]:
results=cursor1.fetchmany(size=3)

In [100]:
print(results)

[(1, 12, 'Anna Iversen', datetime.timedelta(seconds=68400), 1, 'Mario Gollini', 'Manager', '724, Parsley Lane, Old Town, Chicago, IL', 351258074, 'Mario.g@littlelemon.com', '$70,000'), (2, 12, 'Joakim Iversen', datetime.timedelta(seconds=68400), 1, 'Mario Gollini', 'Manager', '724, Parsley Lane, Old Town, Chicago, IL', 351258074, 'Mario.g@littlelemon.com', '$70,000'), (3, 19, 'Vanessa McCarthy', datetime.timedelta(seconds=54000), 3, 'Giorgos Dioudis', 'Head Chef', '879 Sage Streey, West Loop, Chicago, IL', 351970582, 'Giorgos.d@littlelemon.com', '$50,000')]


In [102]:
len(results)

3

In [103]:
guest1.close()